# Challenge Data Science - Semana 1

Na reunião inicial com as pessoas responsáveis pela área de vendas da empresa, foi explicada a importância de se reduzir a Taxa de Evasão de Clientes, conhecido como Churn Rate (quanto a empresa perdeu de receita ou clientes em um período de tempo).

Através da API da Alura Voz, teremos acesso à base de dados e às suas informações.

Na primeira semana sera feito uma analise dos dados para verificação de inconsistencias e sera feito correção delas caso encontrar algo

# Importando Base de Dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
dados = pd.read_json('dados/Telco-Customer-Churn.json')

In [3]:
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [4]:
print("Quantidades de linhas e colunas: ", dados.shape)

Quantidades de linhas e colunas:  (7267, 6)


Constatamos que as colunas de **customer, phone, internet, account** estão com vários valores dentro de chaves, e dificulta a analise dos dados, precisamos expandir essas chaves e transforma-las em novas colunas no nosso dataframe

### Normalizando os dados

In [5]:
customer_dados = pd.json_normalize(data=dados.customer)
customer_dados.head()

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9
2,Male,0,No,No,4
3,Male,1,Yes,No,13
4,Female,1,Yes,No,3


In [6]:
phone_dados = pd.json_normalize(data=dados.phone)
phone_dados.head()

,PhoneService,MultipleLines
0,Yes,No
1,Yes,Yes
2,Yes,No
3,Yes,No
4,Yes,No


In [7]:
internet_dados = pd.json_normalize(data=dados.internet)
internet_dados.head()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,DSL,No,Yes,No,Yes,Yes,No
1,DSL,No,No,No,No,No,Yes
2,Fiber optic,No,No,Yes,No,No,No
3,Fiber optic,No,Yes,Yes,No,Yes,Yes
4,Fiber optic,No,No,No,Yes,Yes,No


In [8]:
account_dados = pd.json_normalize(data=dados.account, sep='_')
account_dados.head()

,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,One year,Yes,Mailed check,65.6,593.3
1,Month-to-month,No,Mailed check,59.9,542.4
2,Month-to-month,Yes,Electronic check,73.9,280.85
3,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Month-to-month,Yes,Mailed check,83.9,267.4


Com todos os valores que estavam em chaves normalizados podemos juntar tudo em um unico dataframe com a função **concat()**

### Juntando todas as normalizações

In [9]:
dados = pd.concat([dados[['customerID', 'Churn']], customer_dados, phone_dados, internet_dados, account_dados], axis=1)
dados.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Agora que temos o dataframe normalizado podemos fazer uma análise para entender as informações que temos e procurar por inconsistencias e possiveis problemas nos dados

## Análise exploratoria dos dados

In [10]:
dados

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


Podemos utlizar um dicionario que a empresa disponibizou para poder endenter mais detalhadamente o significado de cada coluna

### Dicionário de dados
* __customerID:__ número de identificação único de cada cliente
* __Churn:__ se o cliente deixou ou não a empresa
* __gender:__ gênero (masculino e feminino)
* __SeniorCitizen:__ informação sobre um cliente ter ou não idade igual ou maior que 65 anos
* __Partner:__ se o cliente possui ou não um parceiro ou parceira
* __Dependents:__ se o cliente possui ou não dependentes
* __tenure:__ meses de contrato do cliente
* __PhoneService:__ assinatura de serviço telefônico
* __MultipleLines:__ assisnatura de mais de uma linha de telefone
* __InternetService:__ assinatura de um provedor internet
* __OnlineSecurity:__ assinatura adicional de segurança online
* __OnlineBackup:__ assinatura adicional de backup online
* __DeviceProtection:__ assinatura adicional de proteção no dispositivo
* __TechSupport:__ assinatura adicional de suporte técnico, menos tempo de espera
* __StreamingTV:__ assinatura de TV a cabo
* __StreamingMovies:__ assinatura de streaming de filmes
* __Contract:__ tipo de contrato
* __PaperlessBilling:__ se o cliente prefere receber online a fatura
* __PaymentMethod:__ forma de pagamento
* __Charges.Monthly:__ total de todos os serviços do cliente por mês
* __Charges.Total:__ total gasto pelo cliente

In [11]:
print("Quantidades de linhas e colunas: ", dados.shape)

Quantidades de linhas e colunas:  (7267, 21)


In [12]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


Vamos utilizar o método value_counts() em cada uma das colunas para indentificar possíveis categorias com nomes incorretos ou incoerentes.

In [13]:
for coluna in dados.columns:
    print('---------')
    print(f'Numero de dados unicos: {dados[coluna].nunique()}')
    print(f'Dados unicos: {dados[coluna].unique()}')
    print(dados[coluna].value_counts())
    print(f'Tipo da coluna: {dados[coluna].dtype}')

---------
Numero de dados unicos: 7267
Dados unicos: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
0002-ORFBO    1
6614-VBEGU    1
6637-KYRCV    1
6635-MYYYZ    1
6635-CPNUN    1
             ..
3374-TTZTK    1
3374-PZLXD    1
3374-LXDEV    1
3373-YZZYM    1
9995-HOTOH    1
Name: customerID, Length: 7267, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 3
Dados unicos: ['No' 'Yes' '']
No     5174
Yes    1869
        224
Name: Churn, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos: ['Female' 'Male']
Male      3675
Female    3592
Name: gender, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos: [0 1]
0    6085
1    1182
Name: SeniorCitizen, dtype: int64
Tipo da coluna: int64
---------
Numero de dados unicos: 2
Dados unicos: ['Yes' 'No']
No     3749
Yes    3518
Name: Partner, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos:

### Relatorio da Analise dos Dados

- Observando a coluna _customerID_ podemos concluir que não há nenhum ID duplicado e podemos dizer que não há linhas duplicadas no dataset

- Na coluna _Churn_ temos 224 dados não nomeados que precisam ser melhor analisados

- Segundo o dicionario a coluna _SeniorCitzen_ contem informações sobre o cliente ter idade igual ou maior de 65 anos, e pelo padrão adotado por outras colunas podemos substituir os valores (0 1) por (Yes' 'No') para manter o padrão do dataset

- Em Charges.Total tambem temos dados não faltantes e tambem podemos ver que o tipo da coluna é object e devemos mudar para float

## Corrigindo as incosistências nos dados

#### Analisando a coluna _Churn_

In [14]:
dados.loc[dados['Churn']== '']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,9840-GSRFX,,Female,0,No,No,14,Yes,Yes,DSL,...,Yes,No,No,No,No,One year,Yes,Mailed check,54.25,773.2
7180,9872-RZQQB,,Female,0,Yes,No,49,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),40.65,2070.75
7211,9920-GNDMB,,Male,0,No,No,9,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.25,684.85
7239,9955-RVWSC,,Female,0,Yes,Yes,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.9


Analisando o dataset não podemos achar um motivo para a ausencia dos dados (provavelmente foi algum erro estrutural que ocasionou a falta desses dados)

Para não ter problemas nas analises futuras do nosso dataset iremos remover esses dados

In [15]:
dados = dados.loc[dados['Churn'] != ''].reset_index(drop=True)

In [16]:
dados['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

### Analisando _SeniorCitzen_

**Na hora de fazer analises com os dados foi constatado que era melhor usar valores numericos aqui então vou comentar por enquanto**

Iremos substuir os valores para manter o padão do resto das colunas do dataset

In [17]:
dados['SeniorCitizen'].value_counts()

0    5901
1    1142
Name: SeniorCitizen, dtype: int64

Sera criado um dicionario que contem as chaves com os valores que serão trocados e depois sera itilizado a função replace() 

In [18]:
# yes_no = {
#     0 : 'No',
#     1 : 'Yes'
# }

In [19]:
# dados.loc[:, 'SeniorCitizen'] = dados['SeniorCitizen'].replace(yes_no)

In [20]:
dados['SeniorCitizen'].value_counts()

0    5901
1    1142
Name: SeniorCitizen, dtype: int64

### Analisando _Charges.Total_ 

In [21]:
dados.loc[dados['Charges_Total'] == ' ']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
945,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1731,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1906,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2025,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2176,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,
2250,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,
2855,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,
3052,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,
3118,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,
4054,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,


Usando o dicionario podemos observar que a coluna _Charges.Total_ tem uma relação com as colunas _Charges.Monthly_ e _tenure_.

 - A coluna __tenure__ representa a quantidade de meses em que o cliente estava assinando o serviço. 
 - A coluna __Charges.Monthly__ representa os gastos mensais do cliente 
 - __Charges.Total__ é a quantidade total de gastos, portanto seria uma multiplicação de Charges.Monthly por tenure.

Quando tenure == 0, os dados de Charges.Total é vazio (' ')

Vamos selecionar todas as linhas onde o _tenure_ == 0, ou seja, os clientes que assinaram o serviço por 0 meses, e mostrar os resultados para as colunas _Charges.Total_ e _Charges.Monthly._

In [22]:
dados.loc[dados['tenure'] == 0, ['tenure', 'Charges_Monthly', 'Charges_Total']]

,tenure,Charges_Monthly,Charges_Total
945,0,56.05,
1731,0,20.00,
1906,0,61.90,
2025,0,19.70,
2176,0,20.25,
2250,0,25.35,
2855,0,73.35,
3052,0,25.75,
3118,0,52.55,
4054,0,80.85,


Percebe-se que todas as linhas de __Charges.Total__ que estão vazias é porque o cliente não assinou nem por um mês. Temos que preencher esse valor com o mesmo valor que está presente em __Charges.Monthly__ pois esse representa o total.

In [23]:
idx = dados.loc[dados['Charges_Total'] == ' '].index
dados.loc[idx, "Charges_Total"] = dados.loc[idx, "Charges_Monthly"]
dados.loc[dados['tenure'] == 0, ['tenure', 'Charges_Monthly', 'Charges_Total']]

,tenure,Charges_Monthly,Charges_Total
945,0,56.05,56.05
1731,0,20.00,20.0
1906,0,61.90,61.9
2025,0,19.70,19.7
2176,0,20.25,20.25
2250,0,25.35,25.35
2855,0,73.35,73.35
3052,0,25.75,25.75
3118,0,52.55,52.55
4054,0,80.85,80.85


In [24]:
dados['Charges_Total'].value_counts()

20.2      11
19.75      9
19.65      8
20.05      8
19.9       8
          ..
2058.5     1
829.55     1
625.65     1
462.8      1
3707.6     1
Name: Charges_Total, Length: 6541, dtype: int64

E tambem precisamos mudar o tipo da coluna para float

In [25]:
dados['Charges_Total'] = dados['Charges_Total'].astype('float64')

In [26]:
for coluna in dados.columns:
    print('---------')
    print(f'Numero de dados unicos: {dados[coluna].nunique()}')
    print(f'Dados unicos: {dados[coluna].unique()}')
    print(dados[coluna].value_counts())
    print(f'Tipo da coluna: {dados[coluna].dtype}')

---------
Numero de dados unicos: 7043
Dados unicos: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
0002-ORFBO    1
6616-AALSR    1
6625-UTXEW    1
6625-IUTTT    1
6625-FLENO    1
             ..
3352-RICWQ    1
3352-ALMCK    1
3351-NQLDI    1
3351-NGXYI    1
9995-HOTOH    1
Name: customerID, Length: 7043, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos: ['No' 'Yes']
No     5174
Yes    1869
Name: Churn, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos: ['Female' 'Male']
Male      3555
Female    3488
Name: gender, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos: [0 1]
0    5901
1    1142
Name: SeniorCitizen, dtype: int64
Tipo da coluna: int64
---------
Numero de dados unicos: 2
Dados unicos: ['Yes' 'No']
No     3641
Yes    3402
Name: Partner, dtype: int64
Tipo da coluna: object
---------
Numero de dados unicos: 2
Dados unicos: ['Yes' 'No']
N

## Tradução das colunas

In [27]:
dados.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges_Monthly',
       'Charges_Total'],
      dtype='object')

In [28]:
renomear = {
    'customerID' : 'Cliente_ID',
    'Churn' : 'Evasao',
    'gender': 'Genero',
    'SeniorCitizen' : 'Idoso',
    'Partner' : 'Companheiro',
    'Dependents' : 'Dependentes',
    'tenure' : 'meses',
    'PhoneService' : 'Servico_Telefonico',
    'MultipleLines' : 'Multiplas_Linhas',
    'InternetService' : 'Provedor_Internet',
    'OnlineSecurity' : 'Seguranca_Online',
    'OnlineBackup' : 'Backup_Online',
    'DeviceProtection' : 'Protecao_de_Dispositivo',
    'TechSupport' : 'Suporte_Tecnico',
    'StreamingTV' : 'TV_a_Cabo',
    'StreamingMovies' : 'Streaming_de_Filmes',
    'Contract' : 'Contrato',
    'PaperlessBilling' : 'Fatura_Online',
    'PaymentMethod' : 'Metodo_de_Pagamento',
    'Charges_Monthly' : 'Valor_Mensal',
    'Charges_Total' : 'Valor_Total'
}

In [29]:
dados.rename(columns = renomear, inplace = True)
dados.head()

,Cliente_ID,Evasao,Genero,Idoso,Companheiro,Dependentes,meses,Servico_Telefonico,Multiplas_Linhas,Provedor_Internet,...,Backup_Online,Protecao_de_Dispositivo,Suporte_Tecnico,TV_a_Cabo,Streaming_de_Filmes,Contrato,Fatura_Online,Metodo_de_Pagamento,Valor_Mensal,Valor_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [30]:
for coluna in dados.columns[1:]:
    print(dados[coluna].unique())

['No' 'Yes']
['Female' 'Male']
[0 1]
['Yes' 'No']
['Yes' 'No']
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]
['Yes' 'No']
['No' 'Yes' 'No phone service']
['DSL' 'Fiber optic' 'No']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['One year' 'Month-to-month' 'Two year']
['Yes' 'No']
['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
[65.6  59.9  73.9  ... 91.75 68.8  67.85]
[ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]


In [31]:
renomear_variaveis = {
    'No' : 'Nao',
    'Yes' : 'Sim',
    'Female' : 'Feminino',
    'Male' : 'Masculino',
    'No phone service' : 'Sem servico telefonico',
    'DSL' : 'DSL',
    'Fiber optic' : 'Fibra optica',
    'No internet service' : 'Sem servico de internet',
    'One year' : 'Um ano',
    'Month-to-month' : 'Mensal',
    'Two year' : 'Dois anos',
    'Mailed check' : 'Cheque via correio',
    'Electronic check' :'Cheque electrónico',
    'Credit card (automatic)' : 'Cartao de credito (automatico)',
    'Bank transfer (automatic)' : 'Transferencia bancaria (automatico)'
}

In [32]:
dados.replace(renomear_variaveis, inplace=True)
dados

,Cliente_ID,Evasao,Genero,Idoso,Companheiro,Dependentes,meses,Servico_Telefonico,Multiplas_Linhas,Provedor_Internet,...,Backup_Online,Protecao_de_Dispositivo,Suporte_Tecnico,TV_a_Cabo,Streaming_de_Filmes,Contrato,Fatura_Online,Metodo_de_Pagamento,Valor_Mensal,Valor_Total
0,0002-ORFBO,Nao,Feminino,0,Sim,Sim,9,Sim,Nao,DSL,...,Sim,Nao,Sim,Sim,Nao,Um ano,Sim,Cheque via correio,65.60,593.30
1,0003-MKNFE,Nao,Masculino,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Nao,Sim,Mensal,Nao,Cheque via correio,59.90,542.40
2,0004-TLHLJ,Sim,Masculino,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Nao,Sim,Nao,Nao,Nao,Mensal,Sim,Cheque electrónico,73.90,280.85
3,0011-IGKFF,Sim,Masculino,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Sim,Nao,Sim,Sim,Mensal,Sim,Cheque electrónico,98.00,1237.85
4,0013-EXCHZ,Sim,Feminino,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Nao,Sim,Sim,Nao,Mensal,Sim,Cheque via correio,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Nao,Feminino,0,Nao,Nao,13,Sim,Nao,DSL,...,Nao,Nao,Sim,Nao,Nao,Um ano,Nao,Cheque via correio,55.15,742.90
7039,9992-RRAMN,Sim,Masculino,0,Sim,Nao,22,Sim,Sim,Fibra optica,...,Nao,Nao,Nao,Nao,Sim,Mensal,Sim,Cheque electrónico,85.10,1873.70
7040,9992-UJOEL,Nao,Masculino,0,Nao,Nao,2,Sim,Nao,DSL,...,Sim,Nao,Nao,Nao,Nao,Mensal,Sim,Cheque via correio,50.30,92.75
7041,9993-LHIEB,Nao,Masculino,0,Sim,Sim,67,Sim,Nao,DSL,...,Nao,Sim,Sim,Nao,Sim,Dois anos,Nao,Cheque via correio,67.85,4627.65


In [33]:
for coluna in dados.columns[1:]:
    print(dados[coluna].unique())

['Nao' 'Sim']
['Feminino' 'Masculino']
[0 1]
['Sim' 'Nao']
['Sim' 'Nao']
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]
['Sim' 'Nao']
['Nao' 'Sim' 'Sem servico telefonico']
['DSL' 'Fibra optica' 'Nao']
['Nao' 'Sim' 'Sem servico de internet']
['Sim' 'Nao' 'Sem servico de internet']
['Nao' 'Sim' 'Sem servico de internet']
['Sim' 'Nao' 'Sem servico de internet']
['Sim' 'Nao' 'Sem servico de internet']
['Nao' 'Sim' 'Sem servico de internet']
['Um ano' 'Mensal' 'Dois anos']
['Sim' 'Nao']
['Cheque via correio' 'Cheque electrónico'
 'Cartao de credito (automatico)' 'Transferencia bancaria (automatico)']
[65.6  59.9  73.9  ... 91.75 68.8  67.85]
[ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]


## Criar coluna de contas diárias

Crie uma nova coluna contendo os gastos diários do(a) cliente e insira na posição 18 do Data Frame.

In [34]:
dados.insert(19, 'Valor_Diario', (dados['Valor_Mensal'] / 30).round(2))
dados

,Cliente_ID,Evasao,Genero,Idoso,Companheiro,Dependentes,meses,Servico_Telefonico,Multiplas_Linhas,Provedor_Internet,...,Protecao_de_Dispositivo,Suporte_Tecnico,TV_a_Cabo,Streaming_de_Filmes,Contrato,Fatura_Online,Metodo_de_Pagamento,Valor_Diario,Valor_Mensal,Valor_Total
0,0002-ORFBO,Nao,Feminino,0,Sim,Sim,9,Sim,Nao,DSL,...,Nao,Sim,Sim,Nao,Um ano,Sim,Cheque via correio,2.19,65.60,593.30
1,0003-MKNFE,Nao,Masculino,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Sim,Mensal,Nao,Cheque via correio,2.00,59.90,542.40
2,0004-TLHLJ,Sim,Masculino,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Sim,Nao,Nao,Nao,Mensal,Sim,Cheque electrónico,2.46,73.90,280.85
3,0011-IGKFF,Sim,Masculino,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Nao,Sim,Sim,Mensal,Sim,Cheque electrónico,3.27,98.00,1237.85
4,0013-EXCHZ,Sim,Feminino,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Sim,Sim,Nao,Mensal,Sim,Cheque via correio,2.80,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Nao,Feminino,0,Nao,Nao,13,Sim,Nao,DSL,...,Nao,Sim,Nao,Nao,Um ano,Nao,Cheque via correio,1.84,55.15,742.90
7039,9992-RRAMN,Sim,Masculino,0,Sim,Nao,22,Sim,Sim,Fibra optica,...,Nao,Nao,Nao,Sim,Mensal,Sim,Cheque electrónico,2.84,85.10,1873.70
7040,9992-UJOEL,Nao,Masculino,0,Nao,Nao,2,Sim,Nao,DSL,...,Nao,Nao,Nao,Nao,Mensal,Sim,Cheque via correio,1.68,50.30,92.75
7041,9993-LHIEB,Nao,Masculino,0,Sim,Sim,67,Sim,Nao,DSL,...,Sim,Sim,Nao,Sim,Dois anos,Nao,Cheque via correio,2.26,67.85,4627.65


## Armazenar os dados tratados em um novo arquivo

In [35]:
dados.to_json("dados/Telco-Customer-Churn-limpeza.json")  